## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,2022-04-15 18:08:19,-33.6500,115.3333,14.50,91,18,3.00,few clouds
1,1,Harbour Breton,CA,2022-04-15 18:08:19,47.4833,-55.7983,3.87,75,70,2.28,broken clouds
2,2,Biak,ID,2022-04-15 18:08:20,-0.9131,122.8766,24.92,83,95,0.86,moderate rain
3,3,Luderitz,NaN,2022-04-15 18:08:20,-26.6481,15.1594,15.94,67,0,11.56,clear sky
4,4,Kaitangata,NZ,2022-04-15 18:08:21,-46.2817,169.8464,15.13,57,82,6.10,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature in °C you would like for your trip? "))
max_temp = float(input("What is the maximum temperature in °C you would like for your trip? "))


What is the minimum temperature in Celcius you would like for your trip? 18
What is the maximum temperature in Celcius you would like for your trip? 29


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Biak,ID,2022-04-15 18:08:20,-0.9131,122.8766,24.92,83,95,0.86,moderate rain
6,6,Vaini,TO,2022-04-15 18:08:21,-21.2000,-175.2000,25.09,94,20,1.54,few clouds
7,7,Espumoso,BR,2022-04-15 18:08:21,-28.7247,-52.8497,19.66,51,0,3.09,clear sky
8,8,Kapaa,US,2022-04-15 18:08:22,22.0752,-159.3190,23.99,83,40,6.69,scattered clouds
10,10,Cidreira,BR,2022-04-15 18:08:22,-30.1811,-50.2056,18.78,65,99,3.81,overcast clouds
12,12,San Cristobal,VE,2022-04-15 18:05:48,7.7669,-72.2250,27.10,58,92,2.78,overcast clouds
21,21,Santa Cruz De La Palma,ES,2022-04-15 18:08:26,28.6835,-17.7642,21.19,64,0,6.69,clear sky
23,23,Albany,US,2022-04-15 18:06:17,42.6001,-73.9662,19.21,21,18,2.24,few clouds
25,25,Carutapera,BR,2022-04-15 18:08:27,-1.1950,-46.0200,26.86,88,72,3.12,moderate rain
26,26,Teahupoo,PF,2022-04-15 18:08:28,-17.8333,-149.2667,23.98,72,22,7.60,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                305
City                   305
Country                304
Date                   305
Lat                    305
Lng                    305
Max Temp               305
Humidity               305
Cloudiness             305
Wind Speed             305
Current Description    305
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
import numpy as np
preferred_cities_df['Country'].replace('',np.nan)

new_vacation_cities = preferred_cities_df.dropna()
new_vacation_cities.count()

City_ID                304
City                   304
Country                304
Date                   304
Lat                    304
Lng                    304
Max Temp               304
Humidity               304
Cloudiness             304
Wind Speed             304
Current Description    304
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_vacation_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Biak,ID,24.92,moderate rain,-0.9131,122.8766,
6,Vaini,TO,25.09,few clouds,-21.2000,-175.2000,
7,Espumoso,BR,19.66,clear sky,-28.7247,-52.8497,
8,Kapaa,US,23.99,scattered clouds,22.0752,-159.3190,
10,Cidreira,BR,18.78,overcast clouds,-30.1811,-50.2056,
12,San Cristobal,VE,27.10,overcast clouds,7.7669,-72.2250,
21,Santa Cruz De La Palma,ES,21.19,clear sky,28.6835,-17.7642,
23,Albany,US,19.21,few clouds,42.6001,-73.9662,
25,Carutapera,BR,26.86,moderate rain,-1.1950,-46.0200,
26,Teahupoo,PF,23.98,light rain,-17.8333,-149.2667,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
} 
    
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # set base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
# 15 rows should be droped
hotel_df.replace("",np.nan)
#loop and remove where no hotel name is found
for x in hotel_df.index:
    if hotel_df.loc[x,"Hotel Name"] == "":
        hotel_df.drop(x, inplace = True)

In [27]:
hotel_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 289 entries, 2 to 717
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 289 non-null    object 
 1   Country              289 non-null    object 
 2   Max Temp             289 non-null    float64
 3   Current Description  289 non-null    object 
 4   Lat                  289 non-null    float64
 5   Lng                  289 non-null    float64
 6   Hotel Name           289 non-null    object 
dtypes: float64(3), object(4)
memory usage: 26.2+ KB


In [28]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} </dd>
<dt>Max Temp</dt><dd>{Max Temp} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))